<a href="https://colab.research.google.com/github/leonardo3108/IA368dd/blob/main/exercicios/Aula_9/Aula_9_Queries_Negative_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = 'Leonardo Augusto da Silva Pacheco'
print(f'Meu nome é {nome}')

Meu nome é Leonardo Augusto da Silva Pacheco


# Enunciado

https://github.com/leonardo3108/IA368dd/blob/main/exercicios/Aula_9/enunciado.md

Boa parte do código aproveitado do notebook do Hugo (https://colab.research.google.com/drive/1Xp8HLuU_NxPYFQuEIcpGksaJ0f04FzoZ#scrollTo=BO2rS8jnEjMF)


# Setup

## Instalação de libs

In [1]:
!pip install ftfy
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 117.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.6 MB/s eta 0:00:0

## Importação de libs

In [13]:
import array
import ftfy
import json
import math
import numpy as np
import os
import pandas as pd
from collections import defaultdict, Counter
from pyserini.analysis import Analyzer, get_lucene_analyzer
from tqdm.auto import tqdm

# Preparação dos dados

## Obtenção - TREC-COVID

In [3]:
!wget -nc 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

--2023-05-02 22:33:49--  https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73876720 (70M) [application/zip]
Saving to: ‘trec-covid.zip’

trec-covid.zip      100%[===================>]  70.45M  19.3MB/s    in 4.3s    

2023-05-02 22:33:55 (16.3 MB/s) - ‘trec-covid.zip’ saved [73876720/73876720]



In [4]:
!unzip -o trec-covid.zip

Archive:  trec-covid.zip
   creating: trec-covid/
   creating: trec-covid/qrels/
  inflating: trec-covid/qrels/test.tsv  
  inflating: trec-covid/corpus.jsonl  
  inflating: trec-covid/queries.jsonl  


## Tratamento - corpus

In [15]:
df_corpus = pd.read_json('trec-covid/corpus.jsonl', lines=True)
df_corpus['data'] = (df_corpus['title'] + df_corpus['text']).apply(ftfy.fix_text)
df_corpus = df_corpus.drop(['metadata', 'title', 'text'], axis = 1)
df_corpus.columns = ['id', 'text']
df_corpus

,id,text
0,ug7v899j,Clinical features of culture-proven Mycoplasma...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...
2,ejv2xln0,Surfactant protein-D and pulmonary host defens...
3,2b73a28n,Role of endothelin-1 in lung diseaseEndothelin...
4,9785vg6d,Gene expression in epithelial cells in respons...
...,...,...
171327,7e8r61e7,Can Pediatric COVID-19 Testing Sensitivity Be ...
171328,6jittbis,Heterogeneity and plasticity of porcine alveol...
171329,hi8k8wvb,SARS E protein in phospholipid bilayers: an an...
171330,ma3ndg41,Italian Society of Interventional Cardiology (...


In [7]:
df_corpus = pd.read_json('trec-covid/corpus.jsonl', lines=True)
df_corpus['data'] = (df_corpus['title'] + df_corpus['text']).apply(ftfy.fix_text)
df_corpus = df_corpus.drop(['metadata', 'title', 'text'], axis = 1)
df_corpus
df_corpus.columns = ['passage_id', 'passage']

corpus = []
with open('trec-covid/corpus.jsonl') as fin:
    for i, line in enumerate(fin):
        corpus.append(json.loads(line))

for text in corpus[:10]:
    print(text)

{'_id': 'ug7v899j', 'title': 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia', 'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60

## Queries relevantes

Obtenção a partir do upload manual do arquivo **leonardo_pacheco_1k_generated_queries_20230501.jsonl** pela VM.

In [18]:
df_queries = pd.read_json('leonardo_pacheco_1k_generated_queries_20230501.jsonl', lines=True)
df_queries.columns = ['doc_id', 'relevant_query']
df_queries

,doc_id,relevant_query
0,cb63iyr2,Optimization algorithm for multi-aisle automat...
1,bh717utp,What is the basic reproduction number of the n...
2,4i8pepto,ABCD vaccination guidelines for cats
3,y5h942ar,How does the CLEARSY Safety Platform work for ...
4,6v5niqxv,First documented case of morbillivirus infecti...
...,...,...
995,cwswroh6,Surgical management options for post-prostatec...
996,2yblpbwm,Possible query: What are the transmission rout...
997,1j8ethnx,What is de novo gene and genome synthesis and ...
998,7qbcoo9b,How to transform a noninvasive ventilator into...


# Geração de queries não relevantes

## Criação de índice invertido

In [9]:
class InvertedIndex:
    def __init__(self):
        self.index = {}
        self.n_docs = 0
        self.len_docs = {}

    def tokenize(self, text):
       return Analyzer(get_lucene_analyzer(stemmer='porter')).analyze(text)

    def update_index(self, id_doc, document):
        tokenized_document = self.tokenize(document)
        for token, frequency in Counter(tokenized_document).items():
            self.index.setdefault(
                token, {
                    "id_doc"        : [], 
                    "frequency" : array.array("L", [])
                })['id_doc'].append(id_doc)
            self.index.setdefault(
                token, {
                    "id_doc"        : [], 
                    "frequency" : array.array("L", [])
                })['frequency'].append(frequency)
        self.n_docs += 1
        self.len_docs[id_doc] = len(tokenized_document)

    def __getstate__(self):
        return {
            'index'     : self.index,
            'n_docs'    : self.n_docs,
            'len_docs'  : self.len_docs
        }
    
    def __setstate__(self, state):
        self.index     = state['index']
        self.n_docs    = state['n_docs']
        self.len_docs  = state['len_docs']

    def __reduce__(self):
        return (InvertedIndex, (
            self.index, 
            self.n_docs,
            self.len_docs
            )
        )

In [16]:
inverted_index = InvertedIndex()

for i, row in tqdm(df_corpus.iterrows(), desc='Indexing...'):
    inverted_index.update_index(row['id'], row['text'])

Indexing...: 0it [00:00, ?it/s]

## Classe de busca

In [19]:
class TfIdf:
    def __init__(self, inverted_index, k1=1.2, b=0.75):
        self.k1 = k1
        self.b = b
        self.inverted_index = inverted_index
        self.tokenizer = Analyzer(get_lucene_analyzer(stemmer='porter')).analyze 
        self.doc_lengths = self.inverted_index.len_docs
        self.avg_doc_length = np.mean(list(self.doc_lengths.values()))
        self.N = self.inverted_index.n_docs
        
    def score(self, query, hits=1000):
        scored_docs = {}
        for token in self.tokenizer(query):
            if token in self.inverted_index.index:
                # Retrieving document ids, document lengths and frequencies
                id_docs     = np.array(self.inverted_index.index[token]['id_doc'])
                doc_lengths = np.array([self.doc_lengths[doc] for doc in id_docs])
                f           = np.array(self.inverted_index.index[token]['frequency'])

                # Getting number of docs and its inverse frequencies
                n = len(id_docs)
                idf = math.log((self.N - n + 0.5) / (n + 0.5))
                
                # Calculating term frequencies
                tf = ((self.k1 + 1) * f) / (self.k1 * ((1 - self.b) + self.b * (doc_lengths / self.avg_doc_length)) + f)
                    
                # Final score
                scores = idf * tf

                # Appending in an output list
                for id_doc, score in zip(id_docs, scores):
                    scored_docs[id_doc] = scored_docs.get(id_doc, 0) + score

        return sorted(list(scored_docs.items()), key=lambda x: x[1], reverse=True)[:hits]

In [20]:
searcher = TfIdf(inverted_index)

## Execução das queries

In [21]:
def run_all_queries_bm25(bm25, df_queries):
    # Initializing run
    run = {
        'query_id':   [],
        'passage_id': [],
        'score':      [],
        'rank':       [],
        'Q0':         [],
        'run':        []
    }

    for i, row in tqdm(df_queries.iterrows(), desc='Running queries'):

        # Getting query info
        query_id = i
        query = row['relevant_query']

        # Running BM25
        doc_with_scores = bm25.score(query)

        # Recovering passage ids and scores
        passage_ids = [x[0] for x in doc_with_scores]
        scores      = [x[1] for x in doc_with_scores]

        # appending in run
        run['query_id'].extend([query_id] * 1000)
        run['passage_id'].extend(passage_ids)
        run['score'].extend(scores)
        run['rank'].extend(list(range(1,1001)))
        run['Q0'].extend(['Q0'] * 1000)
        run['run'].extend(['bm25_normal'] * 1000)

    # Final df    
    print('Generating final dataframe')
    df_run = pd.DataFrame(run)
    df_run = df_run[['query_id', 'Q0', 'passage_id', 'rank', 'score', 'run']]
    print(df_run.shape)
    return df_run

In [22]:
df_run = run_all_queries_bm25(searcher, df_queries)

Running queries: 0it [00:00, ?it/s]

Generating final dataframe
(1000000, 6)


In [23]:
df_run

,query_id,Q0,passage_id,rank,score,run
0,0,Q0,cb63iyr2,1,64.944539,bm25_normal
1,0,Q0,nu9vtylg,2,31.930582,bm25_normal
2,0,Q0,7xhyrqfw,3,24.044265,bm25_normal
3,0,Q0,g39giox3,4,23.673674,bm25_normal
4,0,Q0,lbjdfosf,5,23.441208,bm25_normal
...,...,...,...,...,...,...
999995,999,Q0,b4ay5vyp,996,10.801486,bm25_normal
999996,999,Q0,hiohwe2o,997,10.799705,bm25_normal
999997,999,Q0,cabnh61m,998,10.793567,bm25_normal
999998,999,Q0,c7q4fp0r,999,10.792723,bm25_normal


## Extração dos docs irrelevantes e gravação do arquivo final

In [27]:
with open('leonardo_pacheco_1k_generated_queries_20230502.jsonl', 'w') as f:
    for i, row in df_queries.iterrows():
        positive_doc_id = row['doc_id']
        query = row['relevant_query']
        negative_doc_ids_list = df_run[df_run['query_id'] == i].sample(n=5, random_state = 10)['passage_id'].tolist()
        line = {
            "positive_doc_id": positive_doc_id, 
            "query": query, 
            "negative_doc_ids": negative_doc_ids_list
        }
        json.dump(line, f)
        f.write("\n")